In [251]:
import pandas as pd
from datetime import datetime
import numpy as np
import json
import glob
import keras
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import json
import seaborn as sns
%matplotlib inline

- Clean notebook to be only new 
- Be able to merge train features and train targets
- same with test
- be able to then split and run program
- test for overfit
- create additional features (business/extra features from swing/ extra features from bugs)
- run reviews for NLP

In [252]:
list_of_file_names = glob.glob("./zip_2/*.csv")

In [253]:
read_data_list_2 = []
fail_list = []
for i, data in enumerate(list_of_file_names):
    try:
        data = pd.read_csv(data)
        #data = unicode(data, errors='replace')
        read_data_list_2.append(data)
    except UnicodeDecodeError:
        fail_list.append(i)
#read_data_list
#fail_list
list_of_file_names_org = list_of_file_names

In [254]:
fail_list

[3]

In [255]:
for x in fail_list:
    del list_of_file_names[x]

In [256]:
list_of_file_names[3]

'./zip_2/Zip_Listings_PriceCut_SeasAdj_AllHomes.csv'

In [315]:
# build loop for data we can work with. Only dates and zip
def build_useful_df(data_list):
    useful_df = []
    for dataset in range(len(data_list)):
        data = read_data_list_2[dataset]
        region_name = data['RegionName']
        region_name = region_name.astype(str)
        new_df = data.select_dtypes(include=['float64'])
        new_df.insert(loc=0, column='RegionName', value=region_name) 
        new_df = new_df.sort_values('RegionName',ascending=False)
        new_df = new_df.set_index('RegionName')
        useful_df.append(new_df)
    return useful_df

In [316]:
useful_df = build_useful_df(read_data_list_2)


In [317]:
sixteen_list = []
for i, file in enumerate(useful_df):
    if len(file) >= 10000:
        sixteen_list.append(i)

In [318]:
len(sixteen_list)

30

In [319]:
len(useful_df)
for num in range(len(useful_df)):
    useful_df[num].dropna(inplace=True)

In [320]:
sixteen_good = [3, 14, 26, 53, 54, 55, 58, 66, 69, 73, 82]

In [321]:
target_data = useful_df[23]

In [322]:
final_data_list = []
for i, data in enumerate(useful_df):
    if i in sixteen_good:
        final_data_list.append(data)

In [323]:
# Docs that don't need feature engineering. They are already aggregates
pure_features = [0, 71, 75, 77, 79, 81]
useful_df[77].head()

,MoM,QoQ,YoY
RegionName,,,
99705,-0.007928,-0.007928,0.019197
99669,0.002829,0.021614,0.012134
99654,-0.015574,-0.036213,-0.074436
99645,-0.010256,-0.040398,-0.093365
99623,-0.002005,-0.007314,-0.042949


In [365]:
def make_features(df, past_time_string, now_string):
    #df.dropna(inplace=True)
    features = pd.DataFrame()
    mean = df.loc[:, past_time_string : now_string].mean(axis=1)
    features['mean'] = mean
    std = df.loc[:, past_time_string : now_string].std(axis=1)
    features['std'] = std
    mn = df.loc[:, past_time_string : now_string].min(axis=1)
    features['min'] = mn
    mx = df.loc[:, past_time_string : now_string].max(axis=1)
    features['max'] = mx
    features['swing'] = mx - mn
    change = df[now_string] - df[past_time_string]
    features['change'] = change
    mean_swing = features['swing'].mean()
    features['swing_pos'] = np.where(features['swing']>mean_swing, 1, 0)
    big_swing = features['swing'].std() + mean_swing
    features['swing_big'] = np.where(features['swing']>big_swing, 1, 0)
    features['swing_neg'] = np.where(features['swing']<mean_swing, 1, 0)
    swing_big_loss = mean_swing - features['swing'].std() 
    features['swing_loss_big'] = np.where(features['swing']<swing_big_loss, 1, 0)
    #features.dropna(inplace=True)
    #features = features.set_index(df.index)
    #print(features.iloc[1, :])
    return features
    
# List of data frames only on one now_time
def make_modeling_data(df_list, df_for_target, now_string):
    df_one = pd.DataFrame()
    target = pd.DataFrame()
    now_time = pd.to_datetime(now_string)
    now_value = df_for_target[now_string]
    future_time = now_time + timedelta(days=6*31)
    future_time_string= future_time.strftime("%Y-%m")
    future_value = df_for_target[future_time_string]
    target['target'] = future_value/now_value
    
    
    past_time = now_time - timedelta(days=6*31)
    past_time_string= past_time.strftime("%Y-%m")
    df_to_use_for_features_org= df_list[0].loc[:, :now_string]
    features_org = make_features(df_to_use_for_features_org, past_time_string, now_string)
    df_one = pd.merge(df_one, features_org, left_index=True, right_index=True, how = 'right')
    for i, df in enumerate(df_list[1:]):
        ind = str(i)
        columns = df.columns
        if '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
            df_to_use_for_features= df.loc[:, :now_string]
            features = make_features(df_to_use_for_features, past_time_string, now_string)
            df_one = df_one.append(features)
            
    #now_time = pd.to_datetime(now_string)
    #now_value = df_for_target[now_string]
    #future_time = now_time + timedelta(days=6*31)
    #future_time_string= future_time.strftime("%Y-%m")
    #future_value = df_for_target[future_time_string]
            #target = future_value/now_value

    
    return df_one, target




test_features, test_targets = make_modeling_data(final_data_list, target_data, "2017-06")
train_features, train_targets = make_modeling_data(final_data_list, target_data, "2016-12")
extra_train_features_1, extra_train_targets_1 = make_modeling_data(final_data_list, target_data, '2016-06')
train_features = train_features.append(extra_train_features_1)
train_targets = train_targets.append(extra_train_targets_1)
#df = useful_df[num]
#columns = df.columns
date_counter = 0
for year in ["2011", "2012", "2013", "2014", "2015"]:
    for month in ["06","12"]:
            new_time = year+"-"+month
            date_counter += 1
            #if '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
                 #   extra_train_features, extra_train_targets = make_modeling_data(df, new_time)
                  #  train_features = train_features.append(extra_train_features)
                   # train_targets = train_targets.append(extra_train_targets)
            extra_train_features, extra_train_targets = make_modeling_data(final_data_list, target_data, new_time)
            extra_train_features.dropna(inplace=True)
            train_features = train_features.append(extra_train_features)
            train_targets = train_targets.append(extra_train_targets)

In [367]:
train_features.dropna(inplace=True)
train_targets.dropna(inplace=True)

In [368]:
# The way to extrapolate smaller df. Now must finish. Smaller must hae index like merge column
train_merge = pd.merge(train_features, train_targets, right_index=True, left_index=True, how='inner')

In [369]:
test_merge = pd.merge(test_features, test_targets, right_index=True, left_index=True, how='inner')

In [370]:
train_merge.dropna(inplace=True)

In [371]:
test_merge.dropna(inplace=True)

In [335]:
train_merge.shape

(7471872, 7)

In [43]:
df_yelp_business = df_yelp_business.sort_values('postal_code',ascending=False)
df_yelp_business = df_yelp_business.set_index('postal_code')

In [44]:
df_yelp_business_merge = pd.DataFrame()
df_yelp_business_merge['review_count'] = df_yelp_business['review_count']

In [45]:
df_yelp_business_merge.head()

,review_count
postal_code,
YO22 5LY,4
YO22 5AL,4
YO22 4RG,3
YO22 4NT,3
YO22 4JT,15


In [924]:
df_yelp_business_merge.tail(50)

,review_count,zeroes


In [59]:
def make_features(df):
    features = pd.DataFrame()
    
    features["my_first_feature"] = df.iloc[:,-1]/df.iloc[:,-2]
    features = features.set_index(df.index)
    return features
    
# List of data frames only on one now_time
def make_modeling_data(df_list, df_for_target, now_string):
    df_one = pd.DataFrame()
    for df in df_list:
        columns = df.columns
        if '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
            df_to_use_for_features= df.loc[:, :now_string]
            features = make_features(df_to_use_for_features)
            df_one = pd.merge(df_one, features, left_index=True, right_index=True, how = 'right')
    
    now_time = pd.to_datetime(now_string)
    now_value = df_for_target[now_string]
    future_time = now_time + timedelta(days=6*31)
    future_time_string= future_time.strftime("%Y-%m")
    future_value = df_for_target[future_time_string]
    target = future_value/now_value

    
    return df_one, target




test_features, test_targets = make_modeling_data(useful_df, useful_df[23], "2017-01")
train_features, train_targets = make_modeling_data(useful_df, useful_df[23], "2016-07")
df = useful_df[num]
columns = df.columns
for year in ["2015","2016"]:
    for month in ["01","07"]:
            new_time = year+"-"+month
                #if '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
                 #   extra_train_features, extra_train_targets = make_modeling_data(df, new_time)
                  #  train_features = train_features.append(extra_train_features)
                   # train_targets = train_targets.append(extra_train_targets)
            extra_train_features, extra_targets = make_modeling_data(useful_df, useful_df[23], new_time)
            train_features = train_features.append(extra_train_features)
            train_targets = train_targets.append(extra_train_targets)

In [372]:
X_train = train_merge.iloc[:, :-1].values
y_train = train_merge.iloc[:, -1].values
X_test = test_merge.iloc[:, :-1].values
y_test = test_merge.iloc[:, -1].values

In [359]:
def norm_per_column(X):
    new_array = []
    shape = X.shape
    columns = shape[1]
    for num in range(columns):
        mx = X[:, num].max()
        if mx == 0:
            new_array.append(X[:, num])
        else:
            X[:, num] = X[:, num] / mx
            new_array.append(X[:, num])
    new_array = np.array(new_array)
    new_array = np.transpose(new_array)
    return new_array

In [373]:
X_train = norm_per_column(X_train)
X_test = norm_per_column(X_test)

In [374]:
X_test.shape

(51888, 10)

In [377]:
# Hold overfit model for now
model = Sequential()
model.add(Dense(3000, input_shape=(10,), activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [375]:
model = Sequential()
model.add(Dense(1000, input_shape=(10,), activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [ ]:
model.fit(x=X_train, y=y_train, 
          batch_size=5000, 
          epochs=10, 
          verbose=1, 
          validation_data=(X_test, y_test),
          callbacks=None)

Train on 7471872 samples, validate on 51888 samples
Epoch 1/10
4760000/7471872 [==================>...........] - ETA: 1:22:17 - loss: 0.0070

In [ ]:
#loss = .00001441 (overfit) = .0038

#std = .072